In [ ]:
from PIL import Image
import math

def getStats(coverImage, stegoImage):
    # Open the image files for reading
    cover = Image.open(coverImage)
    stego = Image.open(stegoImage)

    # Convert images to RGB mode
    cover = cover.convert('RGB')
    stego = stego.convert('RGB')

    # Get the pixel data
    coverPixels = list(cover.getdata())
    stegoPixels = list(stego.getdata())

    # Calculate the MSE (Mean Square Error) and PSNR (Peak Signal to Noise Ratio)
    diffSqrTotal = 0
    for i in range(len(coverPixels)):
        diff = sum((c - s)**2 for c, s in zip(coverPixels[i], stegoPixels[i]))
        diffSqrTotal += diff

    mse = float(diffSqrTotal) / float(len(coverPixels) * 3)  # Multiply by 3 for RGB channels
    print("Mean Square Error (MSE): " + str(mse))

    if mse != 0:
        psnr = 10 * (math.log10((255 * 255 * 3) / mse))  # Multiply by 3 for RGB channels
        print("Peak Signal to Noise Ratio (PSNR): " + str(psnr))
    else:
        print("PSNR is infinite for images with zero MSE.")

    # Calculate the SSIM (Structural Similarity Index)
    c1 = (0.01 * 255)**2
    c2 = (0.03 * 255)**2

    mu_x = sum(pixel[0] for pixel in coverPixels) / len(coverPixels)
    mu_y = sum(pixel[0] for pixel in stegoPixels) / len(stegoPixels)

    sigma_x = math.sqrt(sum((pixel[0] - mu_x)**2 for pixel in coverPixels) / len(coverPixels))
    sigma_y = math.sqrt(sum((pixel[0] - mu_y)**2 for pixel in stegoPixels) / len(stegoPixels))

    sigma_xy = sum((c[0] - mu_x) * (s[0] - mu_y) for c, s in zip(coverPixels, stegoPixels)) / len(coverPixels)

    ssim = ((2 * mu_x * mu_y + c1) * (2 * sigma_xy + c2)) / ((mu_x**2 + mu_y**2 + c1) * (sigma_x**2 + sigma_y**2 + c2))
    print("Structural Similarity Index (SSIM): " + str(ssim))


# getStats("Test_Model/cover_img.jpg", "Test_Model/hidden_cover_img.jpg")
getStats("/content/cover_img.jpg", "/content/hidden_cover_img.jpg")

Mean Square Error (MSE): 5.5481828053792315
Peak Signal to Noise Ratio (PSNR): 45.46050853514457
Structural Similarity Index (SSIM): 0.9981791020517194


In [ ]:
!pip install kaggle

# Upload your Kaggle API key
from google.colab import files
uploaded = files.upload()

# Move the uploaded key to the correct directory
import shutil
key_filename = list(uploaded.keys())[0]
!mkdir -p /root/.kaggle
shutil.move(key_filename, '/root/.kaggle/kaggle.json')
!chmod 600 /root/.kaggle/kaggle.json

# Download the dataset using the Kaggle API
!kaggle datasets download -d vishakhasingh22/ds-vs22

# Unzip the dataset
!unzip -q ds-vs22.zip -d ds_vs22

# List the contents of the dataset directory
!ls ds_vs22

Saving kaggle.json to kaggle.json
 99% 763M/772M [00:10<00:00, 67.7MB/s]
100% 772M/772M [00:10<00:00, 78.2MB/s]
cover  stego


In [ ]:
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras import layers, models

# # Define Alice (Encoder) model
# def make_alice_model():
#     model = models.Sequential()
#     model.add(layers.Input(shape=(64, 64, 3)))  # Assuming input images are 64x64x3
#     model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
#     model.add(layers.MaxPooling2D((2, 2)))
#     model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
#     model.add(layers.MaxPooling2D((2, 2)))
#     model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
#     model.add(layers.MaxPooling2D((2, 2)))
#     model.add(layers.Flatten())
#     model.add(layers.Dense(512, activation='relu'))
#     model.add(layers.Dense(256, activation='relu'))
#     model.add(layers.Dense(128, activation='relu'))
#     model.add(layers.Dense(64, activation='relu'))
#     model.add(layers.Dense(32, activation='relu'))
#     model.add(layers.Dense(16, activation='relu'))
#     model.add(layers.Dense(8, activation='relu'))
#     model.add(layers.Dense(4, activation='sigmoid'))  # Output layer for embedding, values between 0 and 1
#     return model

# # Define Bob (Decoder) model
# def make_bob_model():
#     model = models.Sequential()
#     model.add(layers.Input(shape=(4,)))  # Assuming Alice's output for embedding
#     model.add(layers.Dense(8, activation='relu'))
#     model.add(layers.Dense(16, activation='relu'))
#     model.add(layers.Dense(32, activation='relu'))
#     model.add(layers.Dense(64, activation='relu'))
#     model.add(layers.Dense(128, activation='relu'))
#     model.add(layers.Dense(256, activation='relu'))
#     model.add(layers.Dense(512, activation='relu'))
#     model.add(layers.Dense(64*64*3, activation='sigmoid'))  # Output layer for decoding
#     model.add(layers.Reshape((64, 64, 3)))
#     return model

# # Define Eve (Adversary) model
# def make_eve_model():
#     model = models.Sequential()
#     model.add(layers.Input(shape=(64, 64, 3)))  # Assuming input images are 64x64x3
#     model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
#     model.add(layers.MaxPooling2D((2, 2)))
#     model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
#     model.add(layers.MaxPooling2D((2, 2)))
#     model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
#     model.add(layers.MaxPooling2D((2, 2)))
#     model.add(layers.Flatten())
#     model.add(layers.Dense(512, activation='relu'))
#     model.add(layers.Dense(256, activation='relu'))
#     model.add(layers.Dense(128, activation='relu'))
#     model.add(layers.Dense(64, activation='relu'))
#     model.add(layers.Dense(32, activation='relu'))
#     model.add(layers.Dense(16, activation='relu'))
#     model.add(layers.Dense(8, activation='relu'))
#     model.add(layers.Dense(1, activation='sigmoid'))  # Output layer for classification, binary (0 or 1)
#     return model

# # Define loss functions
# binary_cross_entropy = tf.keras.losses.BinaryCrossentropy()

# # Define optimizers
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# # Training step for Alice and Bob
# @tf.function
# def train_step_alice_bob(images, labels):
#     with tf.GradientTape(persistent=True) as tape:
#         alice_output = alice(images, training=True)
#         bob_output = bob(alice_output, training=True)
#         alice_loss = binary_cross_entropy(images, bob_output)
#         bob_loss = binary_cross_entropy(labels, alice_output)

#     alice_gradients = tape.gradient(alice_loss, alice.trainable_variables)
#     optimizer.apply_gradients(zip(alice_gradients, alice.trainable_variables))

#     bob_gradients = tape.gradient(bob_loss, bob.trainable_variables)
#     optimizer.apply_gradients(zip(bob_gradients, bob.trainable_variables))

# # Training step for Eve
# @tf.function
# def train_step_eve(images, labels):
#     with tf.GradientTape() as tape:
#         eve_output = eve(images, training=True)
#         eve_loss = binary_cross_entropy(labels, eve_output)

#     gradients = tape.gradient(eve_loss, eve.trainable_variables)
#     optimizer.apply_gradients(zip(gradients, eve.trainable_variables))

# # Initialize models
# alice = make_alice_model()
# bob = make_bob_model()
# eve = make_eve_model()

# # Train the models (assuming 'train_images' and 'train_labels' are available)
# epochs = 10
# for epoch in range(epochs):
#     for images, labels in train_dataset:  # Assuming 'train_dataset' is prepared using ImageDataGenerator
#         train_step_alice_bob(images, labels)
#         train_step_eve(images, labels)
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define Alice (Encoder) model
def make_alice_model():
    model = models.Sequential()
    # Define your model architecture for Alice
    return model

# Define Bob (Decoder) model
def make_bob_model():
    model = models.Sequential()
    # Define your model architecture for Bob
    return model

# Define Eve (Adversary) model
def make_eve_model():
    model = models.Sequential()
    # Define your model architecture for Eve
    return model

# Define loss functions
binary_cross_entropy = tf.keras.losses.BinaryCrossentropy()

# Define optimizers
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Training step for Alice and Bob
@tf.function
def train_step_alice_bob(images, labels):
    with tf.GradientTape(persistent=True) as tape:
        alice_output = alice(images, training=True)
        bob_output = bob(alice_output, training=True)
        alice_loss = binary_cross_entropy(images, bob_output)
        bob_loss = binary_cross_entropy(labels, alice_output)

    alice_gradients = tape.gradient(alice_loss, alice.trainable_variables)
    optimizer.apply_gradients(zip(alice_gradients, alice.trainable_variables))

    bob_gradients = tape.gradient(bob_loss, bob.trainable_variables)
    optimizer.apply_gradients(zip(bob_gradients, bob.trainable_variables))

# # Training step for Eve
# @tf.function
# def train_step_eve(images, labels):
#     with tf.GradientTape() as tape:
#         eve_output = eve(images, training=True)
#         eve_loss = binary_cross_entropy(labels, eve_output)

#     gradients = tape.gradient(eve_loss, eve.trainable_variables)
#     optimizer.apply_gradients(zip(gradients, eve.trainable_variables))

# Training step for Alice and Bob
# @tf.function
# def train_step_alice_bob(images, labels):
#     with tf.GradientTape(persistent=True) as tape:
#         alice_output = alice(images, training=True)
#         bob_output = bob(alice_output, training=True)
#         alice_loss = binary_cross_entropy(images, bob_output)
#         # Reshape labels to match the shape of alice_output
#         labels_reshaped = tf.reshape(labels, (-1, 64, 64, 3))
#         bob_loss = binary_cross_entropy(labels_reshaped, alice_output)

#     alice_gradients = tape.gradient(alice_loss, alice.trainable_variables)
#     optimizer.apply_gradients(zip(alice_gradients, alice.trainable_variables))

#     bob_gradients = tape.gradient(bob_loss, bob.trainable_variables)
#     optimizer.apply_gradients(zip(bob_gradients, bob.trainable_variables))

# # Training step for Alice and Bob
# @tf.function
# def train_step_alice_bob(images, labels):
#     with tf.GradientTape(persistent=True) as tape:
#         alice_output = alice(images, training=True)
#         bob_output = bob(alice_output, training=True)
#         alice_loss = binary_cross_entropy(images, bob_output)
#         bob_loss = binary_cross_entropy(labels, alice_output)

#     alice_gradients = tape.gradient(alice_loss, alice.trainable_variables)
#     optimizer.apply_gradients(zip(alice_gradients, alice.trainable_variables))

#     bob_gradients = tape.gradient(bob_loss, bob.trainable_variables)
#     optimizer.apply_gradients(zip(bob_gradients, bob.trainable_variables))

# @tf.function
# def train_step_alice_bob(images, labels):
#     with tf.GradientTape(persistent=True) as tape:
#         alice_output = alice(images, training=True)
#         bob_output = bob(alice_output, training=True)
#         alice_loss = binary_cross_entropy(images, bob_output)
#         # Reshape labels to match the shape of alice_output
#         labels_reshaped = tf.reshape(labels, (-1, 64, 64, 3))
#         bob_loss = binary_cross_entropy(labels_reshaped, alice_output)

#     alice_gradients = tape.gradient(alice_loss, alice.trainable_variables)
#     optimizer.apply_gradients(zip(alice_gradients, alice.trainable_variables))

#     bob_gradients = tape.gradient(bob_loss, bob.trainable_variables)
#     optimizer.apply_gradients(zip(bob_gradients, bob.trainable_variables))

# # Training step for Alice and Bob
# @tf.function
# def train_step_alice_bob(images, labels):
#     with tf.GradientTape(persistent=True) as tape:
#         alice_output = alice(images, training=True)
#         bob_output = bob(alice_output, training=True)
#         alice_loss = binary_cross_entropy(images, bob_output)
#         # Reshape labels to match the shape of alice_output
#         labels_reshaped = tf.reshape(labels, tf.shape(alice_output))
#         bob_loss = binary_cross_entropy(labels_reshaped, alice_output)

#     alice_gradients = tape.gradient(alice_loss, alice.trainable_variables)
#     optimizer.apply_gradients(zip(alice_gradients, alice.trainable_variables))

#     bob_gradients = tape.gradient(bob_loss, bob.trainable_variables)
#     optimizer.apply_gradients(zip(bob_gradients, bob.trainable_variables))

# Training step for Alice and Bob
# @tf.function
# def train_step_alice_bob(images, labels):
#     with tf.GradientTape(persistent=True) as tape:
#         alice_output = alice(images, training=True)
#         bob_output = bob(alice_output, training=True)
#         alice_loss = binary_cross_entropy(images, bob_output)
#         # Reshape labels to match the shape of alice_output
#         labels_reshaped = tf.reshape(labels, tf.shape(alice_output))
#         bob_loss = binary_cross_entropy(labels_reshaped, alice_output)

#     alice_gradients = tape.gradient(alice_loss, alice.trainable_variables)
#     optimizer.apply_gradients(zip(alice_gradients, alice.trainable_variables))

#     bob_gradients = tape.gradient(bob_loss, bob.trainable_variables)
#     optimizer.apply_gradients(zip(bob_gradients, bob.trainable_variables))

# Training step for Alice and Bob
# @tf.function
# def train_step_alice_bob(images, labels):
#     with tf.GradientTape(persistent=True) as tape:
#         alice_output = alice(images, training=True)
#         bob_output = bob(alice_output, training=True)
#         alice_loss = binary_cross_entropy(images, bob_output)
#         # Reshape labels to match the shape of alice_output
#         labels_reshaped = tf.reshape(labels, (-1, 1))  # Reshape labels to (?, 1)
#         labels_reshaped = tf.broadcast_to(labels_reshaped, tf.shape(alice_output))  # Broadcast to match alice_output shape
#         bob_loss = binary_cross_entropy(labels_reshaped, alice_output)

#     alice_gradients = tape.gradient(alice_loss, alice.trainable_variables)
#     optimizer.apply_gradients(zip(alice_gradients, alice.trainable_variables))

#     bob_gradients = tape.gradient(bob_loss, bob.trainable_variables)
#     optimizer.apply_gradients(zip(bob_gradients, bob.trainable_variables))

#bard
def train_step_alice_bob(images, labels):
  with tf.GradientTape(persistent=True) as tape:
    alice_output = alice(images, training=True)
    bob_output = bob(alice_output, training=True)
    alice_loss = binary_cross_entropy(images, bob_output)

    # Reshape to match alice_output's first dimension
    labels_reshaped = tf.reshape(labels, (-1, 1))

    # Reshape bob_output if necessary
    # (e.g., tf.squeeze() or custom reshaping depending on bob_output's shape)

    bob_loss = binary_cross_entropy(labels_reshaped, bob_output)

    alice_gradients = tape.gradient(alice_loss, alice.trainable_variables)
    optimizer.apply_gradients(zip(alice_gradients, alice.trainable_variables))

    bob_gradients = tape.gradient(bob_loss, bob.trainable_variables)
    optimizer.apply_gradients(zip(bob_gradients, bob.trainable_variables))


# Create ImageDataGenerator for loading data
datagen = ImageDataGenerator(rescale=1./255)

# Load images from directories
batch_size = 32
train_generator = datagen.flow_from_directory(
    'ds_vs22',
    target_size=(64, 64),  # Assuming input images are resized to 64x64
    batch_size=batch_size,
    class_mode='binary',  # Assuming there are two classes: cover and stego
    shuffle=True
)

# Initialize models
alice = make_alice_model()
bob = make_bob_model()
eve = make_eve_model()

# Train the models
epochs = 10
for epoch in range(epochs):
    for images, labels in train_generator:
        train_step_alice_bob(images, labels)
        train_step_eve(images, labels)


Found 3359 images belonging to 2 classes.


InvalidArgumentError: {{function_node __wrapped__Mul_device_/job:localhost/replica:0/task:0/device:CPU:0}} Incompatible shapes: [32,1] vs. [32,64,64,3] [Op:Mul] name: 

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

# Define Alice (Encoder) model
def make_alice_model():
    model = models.Sequential()
    model.add(layers.Input(shape=(64, 64, 3)))  # Assuming input images are 64x64x3
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(16, activation='relu'))
    model.add(layers.Dense(8, activation='relu'))
    model.add(layers.Dense(4, activation='sigmoid'))  # Output layer for embedding, values between 0 and 1
    return model

# Define Bob (Decoder) model
def make_bob_model():
    model = models.Sequential()
    model.add(layers.Input(shape=(4,)))  # Assuming Alice's output for embedding
    model.add(layers.Dense(8, activation='relu'))
    model.add(layers.Dense(16, activation='relu'))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(64*64*3, activation='sigmoid'))  # Output layer for decoding
    model.add(layers.Reshape((64, 64, 3)))
    return model

# Define loss function
binary_cross_entropy = tf.keras.losses.BinaryCrossentropy()

# Define optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Training step for Alice and Bob
# @tf.function
# def train_step(images, labels):
#     with tf.GradientTape(persistent=True) as tape:
#         alice_output = alice(images, training=True)
#         bob_output = bob(alice_output, training=True)
#         alice_loss = binary_cross_entropy(images, bob_output)
#         bob_loss = binary_cross_entropy(labels, alice_output)

#     alice_gradients = tape.gradient(alice_loss, alice.trainable_variables)
#     optimizer.apply_gradients(zip(alice_gradients, alice.trainable_variables))

#     bob_gradients = tape.gradient(bob_loss, bob.trainable_variables)
#     optimizer.apply_gradients(zip(bob_gradients, bob.trainable_variables))


@tf.function
def train_step(images, labels):
    with tf.GradientTape(persistent=True) as tape:
        alice_output = alice(images, training=True)
        bob_output = bob(alice_output, training=True)
        alice_loss = binary_cross_entropy(images, bob_output)
        bob_loss = binary_cross_entropy(images, alice_output)  # Use images for both input and labels

    alice_gradients = tape.gradient(alice_loss, alice.trainable_variables)
    optimizer.apply_gradients(zip(alice_gradients, alice.trainable_variables))

    bob_gradients = tape.gradient(bob_loss, bob.trainable_variables)
    optimizer.apply_gradients(zip(bob_gradients, bob.trainable_variables))


# Load dataset using ImageDataGenerator
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
train_generator = datagen.flow_from_directory(
        'ds_vs22',
        target_size=(64, 64),
        batch_size=32,
        class_mode=None,  # No labels, just the images
        shuffle=True)  # Shuffle the data for better training

# Initialize models
alice = make_alice_model()
bob = make_bob_model()

# Train the models
epochs = 10
# for epoch in range(epochs):
#     for images in train_generator:
#         train_step(images, images)  # Passing images as both input and labels for autoencoder
#         break  # Only one batch for demonstration, remove break for full training
# Continue training for the remaining batches in the dataset
for epoch in range(epochs):
    for images in train_generator:
        train_step(images, images)  # Passing images as both input and labels for autoencoder

# Optionally, you can save the trained models
alice.save("alice_model.h5")
bob.save("bob_model.h5")


Found 3359 images belonging to 2 classes.


ValueError: in user code:

    File "<ipython-input-15-c1e4eb708953>", line 69, in train_step  *
        bob_loss = binary_cross_entropy(images, alice_output)  # Use images for both input and labels
    File "/usr/local/lib/python3.10/dist-packages/keras/src/losses.py", line 143, in __call__  **
        losses = call_fn(y_true, y_pred)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/losses.py", line 270, in call  **
        return ag_fn(y_true, y_pred, **self._fn_kwargs)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/losses.py", line 2532, in binary_crossentropy
        backend.binary_crossentropy(y_true, y_pred, from_logits=from_logits),
    File "/usr/local/lib/python3.10/dist-packages/keras/src/backend.py", line 5822, in binary_crossentropy
        return tf.nn.sigmoid_cross_entropy_with_logits(

    ValueError: `logits` and `labels` must have the same shape, received ((32, 4) vs (32, 64, 64, 3)).
